<a href="https://colab.research.google.com/github/thisiseshan/style-transfer-tut/blob/master/Style_Transfer_Tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Style Transfer tutorial with Deep Neural Networks

We will use a VGG-19 model to go about this notebook in PyTorch.  Conv_1_1 is the first convolutional layer that an image is passed through, in the first stack. Conv_2_1 is the first convolutional layer in the second stack. The deepest convolutional layer in the network is conv_5_4.

We will first separate our image into two sections:


1.   Content image
2.   Style image

Basic idea is to extract objects and structures from the **Content image** and extract style and colors from **Style image**.

Enough talk, Let's get started!
